In [ ]:
import os
import time
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.restoration import denoise_wavelet
from skimage.feature import local_binary_pattern
import cv2

In [ ]:
# Feature Extration using local binary pattern
def lbp_feature_extraction(im, n_points, radius, method):
    im_gray = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2GRAY)
    lbp = local_binary_pattern(im_gray, n_points, radius, method)
    # Features returned are the hisogram of resulting data from lbp algorithm
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, radius + 2), normed=True)
    return hist

In [ ]:
# Generating train and test path
input_path = Path('../input')
train_path = input_path / 'train'
test_path = input_path / 'test'

In [ ]:
# Listing all directories in trainning path
cameras = os.listdir(train_path)

In [ ]:
# Generating DataFrame of images and labes in the trainning dataset
train_images = []
for camera in cameras:
    for fname in sorted(os.listdir(train_path / camera)):
        train_images.append((camera, fname))
train = pd.DataFrame(train_images, columns=['camera', 'fname'])

In [ ]:
# Generating DataFrame of images and labes in the testing dataset
test_images = []
for fname in sorted(os.listdir(test_path)):
    test_images.append(fname)
test = pd.DataFrame(test_images, columns=['fname'])

In [ ]:
seg = train[train['camera'] == cameras[0]]
seg = seg.reset_index()
path_im = train_path / seg.get_value(0, 'camera') / seg.get_value(80, 'fname')
print(path_im)
bgr_image = cv2.imread(str(path_im))
rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
im = cv2.resize(rgb_image, (100, 50))
im_gray = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2GRAY)

In [ ]:
plt.imshow(im)
plt.figure()
plt.imshow(im_gray, cmap='gray')
radius = 3
n_points = 8
lbp = local_binary_pattern(im_gray, n_points, radius)
plt.figure()
plt.imshow(lbp, cmap='gray')

# Procedure to mount feature vectors
data = []
target = []
m_samples = np.shape(train)[0]
start = time.time()
for camera in cameras:
    print("Feature extraction: %s"%(camera))
    for i in range(5):
        print("Example %i"%i)
        seg = train[train['camera'] == camera]
        seg = seg.reset_index()
        path_im = train_path / seg.get_value(i, 'camera') / seg.get_value(i, 'fname')
        bgr_image = cv2.imread(str(path_im))
        im = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
        data.append(lbp_feature_extraction(im, 24, 8, 'uniform'))
        target.append(seg.get_value(0, 'camera'))
end = time.time()
print('The time elapsed to extract all lbp features was: %f'%(end-start))

print(data, target)

In [ ]:
im_wav = denoise_wavelet(im)
print(np.shape(im_wav))
plt.imshow(im_wav)
noise = im - im_wav
print(np.shape(noise))
print(np.round(noise))
plt.figure()
plt.imshow(np.round(noise))

In [ ]:
a = lbp.ravel()
print(a[:30])

In [ ]:
(fig, ax) = plt.subplots()
ax.hist(lbp.ravel(), normed=True, bins=20, range=(0,256))

In [ ]:
denoise_wavelet

In [ ]:
'''
# Automated applying feature extraction
x = []

print(train.iterrows())

for i, j in train.iterrows():
    print('%s'%(j['camera']))
    print('Extracting feature from image %i'%i)
    path_im = train_path / j['camera'] / j['fname']
    im = Image.open(path_im)
    im_gray = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2GRAY)
    #plt.imshow(im)
    #plt.imshow(im_gray)
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(im_gray, n_points, radius)
    #plt.imshow(lbp, cmap='gray')
    a = lbp.ravel()
    #(fig, ax) = plt.subplots()
    #ax.hist(lbp.ravel(), normed=True, bins=20, range=(0,256))''''''